<a href="https://colab.research.google.com/github/deboraydo/Spoon-Knife/blob/master/notebook/Tutorial03_TWSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>  


# **Water storage changes at the basin scale**  
### **PEER2PEER GEE Training**  
**Developed by:** Debora Yumi de Oliveira<sup>1,2</sup>, Annika Hjelmstad<sup>1</sup>,  Muhammad Umar Akbar<sup>3</sup>, Kasra Khodkar<sup>3</sup>

**Affiliation:**
<sup>1</sup>Department of Civil and Environmental Engineering, University of California at Irvine, Irvine, USA  
<sup>2</sup>Mackenzie Presbyterian University, São Paulo, Brazil  
<sup>3</sup>Department of Biosystems and Agricultural Engineering, Oklahoma State University, Stillwater, USA  

</center>

---

## **Introduction**

Welcome to this Google Earth Engine (GEE) training notebook! In this tutorial, we will work with the [GRACE Monthly Mass Grids](https://developers.google.com/earth-engine/datasets/catalog/NASA_GRACE_MASS_GRIDS_V04_LAND) in the Google Earth Engine data catalog to explore changes in the total terrestrial water storage anomalies. This dataset is produced by three centers: CSR (U. Texas / Center for Space Research), GFZ (GeoForschungsZentrum Potsdam), and JPL (NASA Jet Propulsion Laboratory). We will explore the output provided by each center and compute basin-averaged values using basin boundaries of the [HydroSHEDS database](https://www.hydrosheds.org/).

---

First, we will install all necessary Python modules:

In [ ]:
!pip install geemap matplotlib pandas numpy scipy

Next, we need to import the Python modules we will use and authenticate and initialize with the Google Cloud project we created previously.

In [ ]:
# Import necessary Python modules
import ee                        # ee provides functions for sending requests to the GEE servers
import geemap                    # geemap is for mapping ee objects
import matplotlib.pyplot as plt  # matplotlib is a general Python plotting module
from matplotlib import gridspec
import pandas as pd              # pandas is a Python module for handling dataframes
import numpy as np               # numpy is a Python module for working with arrays and matrices
import scipy.stats               # scipy is a Python module that provides common algorithms in math and science

# Authenticate and initialize with your google cloud project
ee.Authenticate()
ee.Initialize(project='annikas-tutorial')  # replace the project with your own project ID

Now we're ready to read in the data. We use `ee.ImageCollection` to read the monthly dataset.

In [ ]:
# Read monthly water mass anomalies image collection
twsa_monthly = ee.ImageCollection("NASA/GRACE/MASS_GRIDS_V04/LAND")

# Read watersheds feature collection
basins_africa = (ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_2')
    # Filter to Africa
    .filterBounds(ee.Geometry.Rectangle(-17, -35, 52, 25))
    )



# Read CHIRPS daily data and filter for the year 2018
chirps = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
    # .filterDate('2018-01-01', '2018-12-31')
    )

# Select the precipitation band and sum over the year
start_date = '2018-01-01';
end_date = '2018-12-31';
yearly_precip = chirps.select('precipitation').filterDate(start_date, end_date).sum()

In [ ]:
yearly_precip

# 1. Inspect monthly TWSA dataset

Let's see what's in the image collection. (Click on the dropdown arrows after running the following cell.)

In [ ]:
twsa_monthly  # this will print out information about the variable

In [ ]:
def plot_twsa(image, title, existing_map=None):

    if existing_map is None:
        Map = geemap.Map()  # Initialize map object if we don't already have one
    else:
        Map = existing_map
    vis_params = {
        'palette': ['ff0303', 'ffffff', '0300ff'],
        'min': -1, 'max': 1
    }  # Set the color scale for the map
    Map.addLayer(image, vis_params, title, False)  # Add the layer to our map object.
    if existing_map is None:
            Map.add_colorbar(
            vis_params,
            label="Equivalent Water Thickness (cm)",
            orientation="vertical",
            position="bottomleft",
            transparent_bg=True,
        )  # Add legend as a vertical colorbar
    return Map

# Map one month: CSR
csr = ee.Image('NASA/GRACE/MASS_GRIDS_V04/LAND/20170409_20170508').select('lwe_thickness_csr');
Map = plot_twsa(csr, "CSR")

# Map one month: GFZ
gfz = ee.Image('NASA/GRACE/MASS_GRIDS_V04/LAND/20170409_20170508').select('lwe_thickness_gfz');
plot_twsa(gfz, "GFZ", Map)

# Map one month: JPL
jpl = ee.Image('NASA/GRACE/MASS_GRIDS_V04/LAND/20170409_20170508').select('lwe_thickness_jpl');
plot_twsa(jpl, "JPL", Map)

# 2. Inspect HydroBASINS dataset

Let's see what's in the feature collection.

In [ ]:
# Fetch collection metadata (`.limit(0)`). The printed object is a
# dictionary where keys are column names and values are datatypes.
basins_africa.limit(0).getInfo()['columns']

In [ ]:
# Print the number of basins
display('Count:', basins_africa.size())

In [ ]:
# Display selected basins
Map.addLayer(basins_africa, {'color': '808080'}, 'Basins')
Map

# 3. Spatial patterns of water storage

In [ ]:
def get_twsa_mean(image, collection):

    basins = (
            image.reduceRegions(
                collection=basins_africa,
                reducer=ee.Reducer.mean(),
                scale=111320,
                tileScale=1
              )
          )
    return basins

# Map one month: CSR
csr = ee.Image('NASA/GRACE/MASS_GRIDS_V04/LAND/20170409_20170508').select('lwe_thickness_csr');
basins_africa_csr = get_twsa_mean(csr, basins_africa)

# Map one month: GFZ
gfz = ee.Image('NASA/GRACE/MASS_GRIDS_V04/LAND/20170409_20170508').select('lwe_thickness_gfz');
basins_africa_csr = get_twsa_mean(gfz, basins_africa)

# Map one month: JPL
jpl = ee.Image('NASA/GRACE/MASS_GRIDS_V04/LAND/20170409_20170508').select('lwe_thickness_jpl');
basins_africa_jpl = get_twsa_mean(jpl, basins_africa)

In [ ]:
basins_africa_jpl

In [ ]:
# Create an empty image into which to paint the features, cast to byte.
empty = ee.Image().byte()

# Paint all the polygon edges with the same number and width, display.
fills = empty.paint(featureCollection=basins_africa_jpl, color='mean')

vis_params = {
        'palette': ['ff0303', 'ffffff', '0300ff'],
        'min': -0.25, 'max': 0.25
    }  # Set the color scale for the map

Map.addLayer(fills, vis_params, 'TWSA')

# Display map
Map

# 4. Water storage change over time

In [ ]:
#selected_basin = basins_africa.filter('HYBAS_ID == 1020040190') # 1
#selected_basin = basins_africa.filter('HYBAS_ID == 1020000010') # 2
#selected_basin = basins_africa.filter('HYBAS_ID == 1020011530') # 3
#selected_basin = basins_africa.filter('HYBAS_ID == 1020021940') # 4
#selected_basin = basins_africa.filter('HYBAS_ID == 1020027430') # 5
#selected_basin = basins_africa.filter('HYBAS_ID == 1020034170') # 6
#selected_basin = basins_africa.filter('HYBAS_ID == 1020035180') # 7
selected_basin = basins_africa.filter('HYBAS_ID == 2020071190') # 8

# Display selected basin
Map.addLayer(selected_basin, {'color': '808080'}, 'Selected basin')
Map

In [ ]:
def get_twsa_mean(image, geometry, property):
    """Sets TWSA mean as as a property of the image
    ---
    Params:
        image (ee.Image)
        geometry (ee.Geometry):
            Region of interest
    Returns:
        ee.Image:
            Same as the input image, but with a band 'area' equal to the sum of
            the surface water area
    """

    # The input feature collection is returned with new properties appended.
    # The new properties are the outcome of the region reduction per image band,
    # for each feature in the collection. Region reduction property names
    # are the same as the input image band names.
    twsa_mean = (
        image.reduceRegion(
                          reducer=ee.Reducer.mean(),
                          geometry=geometry,
                          scale=111320,
                          tileScale=1
                          )
            .get(property)
    )
    return image.set('date', image.date().format()).set('twsa_mean', twsa_mean)

In [ ]:
def get_values_from_ee(image_collection, geometry, property):
    """ Get ee TWSA time series values into a local Pandas dataframe
    ---
    Params:
        image_collection (ee.ImageCollection)
        roi (ee.Geometry)
    Returns:
        pd.DataFrame:
            Table with a datetime index and TWSA values
    """
    # Apply the get_twsa_mean function to each image
    twsa = image_collection.map(lambda image: get_twsa_mean(image,geometry,property))
    values = twsa.reduceColumns(
        ee.Reducer.toList(2), ['date', 'twsa_mean']
    ).values().get(0)  # Reduces the images properties to a list of lists
    lista = ee.List(values)  # Type casts the result into a List
    water_areas = ee.Dictionary(lista.flatten())  # Converts the list of lists to a Dictionary
    # This is where we call getInfo()
    areas = pd.DataFrame.from_dict(water_areas.getInfo(), orient='index', columns=['twsa_mean'])
    # Now that we have a python object, represent dates as Python datetimes
    areas.index = pd.to_datetime(areas.index)
    return areas

In [ ]:
twsa_csr = get_values_from_ee(twsa_monthly.select('lwe_thickness_csr'),  selected_basin, 'lwe_thickness_csr')
twsa_gfz = get_values_from_ee(twsa_monthly.select('lwe_thickness_gfz'),  selected_basin, 'lwe_thickness_gfz')
twsa_jpl = get_values_from_ee(twsa_monthly.select('lwe_thickness_jpl'),  selected_basin, 'lwe_thickness_jpl')

In [ ]:
from matplotlib import pyplot as plt
twsa_csr['twsa_mean'].plot(kind='line', figsize=(8, 4), label='CSR')
twsa_gfz['twsa_mean'].plot(kind='line', figsize=(8, 4), label='GFZ')
twsa_jpl['twsa_mean'].plot(kind='line', figsize=(8, 4), label='JPL')
plt.ylabel('Basin-averaged TWSA (cm)')
plt.legend()
plt.gca().spines[['top', 'right']].set_visible(False)